# Comparing various outer material configurations for ARF

We look at how including polymer layers of various types influences loss in ARF.  Note that this is mostly for finding location of modes and to give a general idea of changes, not for final determination of loss.  The latter requires convergence studies for each configuration.

In [ ]:
from fiberamp.fiber.microstruct.pbg import ARF2

from ngsolve.webgui import Draw
import ngsolve as ng

# Default case: Air in outer region

In [ ]:
A = ARF2(name='fine_cladding', poly_core=True)

In [ ]:
Draw(A.index, A.mesh)

In [ ]:
center = 5.066
radius = .1        
alpha = 5


betas, zsqrs, E, phi, _ = A.leakyvecmodes(ctr=center, rad=radius, alpha=alpha,
                                          nspan=4, npts=4, p=1, niterations=10,
                                          rhoinv=.95, quadrule='ellipse_trapez_shift',
                                          nrestarts=0, stop_tol=1e-9)


In [ ]:
betas

In [ ]:
for e in E:
    Draw(e.real, A.mesh, vectors={'grid_size':200})

# Only glass everywhere outside core:

Now that our outer refractive index is higher than the core index, the locations of the Z values switches: the beta values are always approximately equal to k_low = k0 * n_low,  but the Z^2 values equal beta^2 - n0^2, and now n0 is drastically different, so we need to make up for this. 

The formula is at the bottom of the next cell

In [ ]:
scaling = 15
n_glass = 1.4388164768221814
n_air = 1.00027717

n_soft_polymer = 1.44
n_hard_polymer = 1.56

T_soft_polymer = 30 / scaling
T_hard_polymer = 30 / scaling

T_outer = 15 / scaling
T_buffer = 10 / scaling

n0 = n_glass

outer_materials = [

#     {'material': 'soft_polymer',
#      'n': n_soft_polymer,
#      'T': T_soft_polymer,
#      'maxh': 2},

#     {'material': 'hard_polymer',
#      'n': n_hard_polymer,
#      'T': T_hard_polymer,
#      'maxh': 2},

#     {'material': 'buffer',
#      'n': n0,
#      'T': T_buffer,
#      'maxh': 2},

    {'material': 'Outer',
     'n': n0,
     'T': T_outer,
     'maxh': .2}
]

center = A.L**2 * A.k**2 * (n0**2 - n_air**2) + 5.066

In [ ]:
center

In [ ]:
A = ARF2(name='fine_cladding', poly_core=True, outer_materials=outer_materials)


In [ ]:
Draw(A.V, A.mesh)

In [ ]:
center = center
radius = .1       
alpha = 5


betas, zsqrs, E, phi, _ = A.leakyvecmodes(ctr=center, rad=radius, alpha=alpha,
                                          nspan=4, npts=4, p=2, niterations=6,
                                          rhoinv=.95, quadrule='ellipse_trapez_shift',
                                          nrestarts=0, stop_tol=1e-9)


In [ ]:
betas

In [ ]:
for e in E:
    Draw(e.real, A.mesh, vectors={'grid_size':200})

# Only soft polymer, extending everywhere outside core and cladding.

In [ ]:
scaling = 15
n_glass = 1.4388164768221814
n_air = 1.00027717

n_soft_polymer = 1.44
n_hard_polymer = 1.56

T_soft_polymer = 10 / scaling
T_hard_polymer = 30 / scaling

T_outer = 15 / scaling
T_buffer = 30 / scaling

n0 = n_soft_polymer

outer_materials = [

    {'material': 'soft_polymer',
     'n': n_soft_polymer,
     'T': T_soft_polymer,
     'maxh': .04},

#     {'material': 'hard_polymer',
#      'n': n_hard_polymer,
#      'T': T_hard_polymer,
#      'maxh': 2},

#     {'material': 'buffer',
#      'n': n0,
#      'T': T_buffer,
#      'maxh': 2},

    {'material': 'Outer',
     'n': n0,
     'T': T_outer,
     'maxh': .2}
]
center = A.L**2 * A.k**2 * (n0**2 - n_air**2) + 5.066

In [ ]:
center

In [ ]:
A = ARF2(name='fine_cladding', poly_core=True, refine=0, outer_materials=outer_materials)


In [ ]:
Draw(A.V, A.mesh)

In [ ]:
 A.mesh.GetMaterials()

In [ ]:
center = center
radius = .1      
alpha = 5


betas, zsqrs, E, phi, _ = A.leakyvecmodes(ctr=center, rad=radius, alpha=alpha,
                                          nspan=4, npts=4, p=1, niterations=6,
                                          rhoinv=.95, quadrule='ellipse_trapez_shift',
                                          nrestarts=0, stop_tol=1e-9)


In [ ]:
betas

In [ ]:
for e in E:
    Draw(e.real, A.mesh, vectors={'grid_size':200})

# Soft and Hard Polymer, Hard polymer extending to infinity

In [ ]:
scaling = 15
n_glass = 1.4388164768221814
n_air = 1.00027717

n_soft_polymer = 1.44
n_hard_polymer = 1.56

T_soft_polymer = 30 / scaling
T_hard_polymer = 15 / scaling

T_outer = 15 / scaling
T_buffer = 30 / scaling

n0 = n_hard_polymer

outer_materials = [

    {'material': 'soft_polymer',
     'n': n_soft_polymer,
     'T': T_soft_polymer,
     'maxh': .06},

    {'material': 'hard_polymer',
     'n': n_hard_polymer,
     'T': T_hard_polymer,
     'maxh': .07},

#     {'material': 'buffer',
#      'n': n0,
#      'T': T_buffer,
#      'maxh': 2},

    {'material': 'Outer',
     'n': n0,
     'T': T_outer,
     'maxh': .2}
]
center = A.L**2 * A.k**2 * (n0**2 - n_air**2) + 5.066

In [ ]:
A = ARF2(name='fine_cladding', poly_core=True, refine=0, outer_materials=outer_materials)


In [ ]:
Draw(A.V, A.mesh)

In [ ]:
 A.mesh.GetMaterials()

In [ ]:
center = center
radius = .1        
alpha = 5


betas, zsqrs, E, phi, _ = A.leakyvecmodes(ctr=center, rad=radius, alpha=alpha,
                                          nspan=4, npts=4, p=2, niterations=10,
                                          rhoinv=.95, quadrule='ellipse_trapez_shift',
                                          nrestarts=0, stop_tol=1e-9)


In [ ]:
betas

In [ ]:
for e in E:
    Draw(e.real, A.mesh, vectors={'grid_size':200})

# Both polymer layers, then air for outer region:

In [ ]:
scaling = 15
n_glass = 1.4388164768221814
n_air = 1.00027717

n_soft_polymer = 1.44
n_hard_polymer = 1.56

T_soft_polymer = 30 / scaling
T_hard_polymer = 30 / scaling

T_outer = 30 / scaling
T_buffer = 10 / scaling

n0 = n_air

outer_materials = [

    {'material': 'soft_polymer',
     'n': n_soft_polymer,
     'T': T_soft_polymer,
     'maxh': .3},

    {'material': 'hard_polymer',
     'n': n_hard_polymer,
     'T': T_hard_polymer,
     'maxh': .4},

    {'material': 'buffer',
     'n': n0,
     'T': T_buffer,
     'maxh': 1},

    {'material': 'Outer',
     'n': n0,
     'T': T_outer,
     'maxh': 2}
]

In [ ]:
A = ARF2( poly_core=True, refine=0, outer_materials=outer_materials)


In [ ]:
Draw(A.index, A.mesh)

In [ ]:
 A.mesh.GetMaterials()

In [ ]:
center = 5.066
radius = .1        
alpha = 5


betas, zsqrs, E, phi, _ = A.leakyvecmodes(ctr=center, rad=radius, alpha=alpha,
                                          nspan=4, npts=4, p=1, niterations=10,
                                          rhoinv=.95, quadrule='ellipse_trapez_shift',
                                          nrestarts=0, stop_tol=1e-9)


In [ ]:
Draw(A.V, A.mesh)

In [ ]:
betas

In [ ]:
for e in E:
    Draw(e.real, A.mesh, vectors={'grid_size':200})